## <center> Buffer Overflow Attack </center>

### Program Memory Layout

When a program runs, it is loaded into memory.

A typical C program divides memory into five segments
- Text segment (also known as code segment): stores the executable code of the program (usually read-only)
- Data segment: stores **static/global** variables initialized in the program. 
  - static char s[] = "hello world";
  - static int a = 1;
- BSS (block stared by symbol) segment: stores uninitialized **static/global** variables. These variables will be initialized with zeros. 
- Heap: provides space for dynamic memory allocation (caused by `malloc`, `calloc`, `realloc`, `free`, etc). 
- Stack: is simple data structure with a LIFO (last-in-first-out access policy). Stack stores local variables defined inside functions, and data related to function calls (return address, arguments, etc)


- Sizes of text, data, and BSS segment are known as soon as compilation or assembly is completed.
- Stack and heap segments will grow and shrink during program execution.
  - Therefore, they tend to be configured such that they grow toward each other. 
  - The boundary between them is flexible.
  - Both can grow until all available memory is used. 

<center> <img src="figure/buffer-overflow/bo1.png" width="400"/>

In [2]:
%%writefile source/mem_layout.c

int x = 100;
int main()
{
  // data stored on stack
  int   a=2;
  float b=2.5;
  static int y;

  // allocate memory on heap
  int *ptr = (int *) malloc(2*sizeof(int));
  
  // values 5 and 6 stored on heap
  ptr[0]=5;
  ptr[1]=6;

  // deallocate memory on heap	
  free(ptr);
 
  return 1;
}

Writing source/mem_layout.c


Run the followings:

```
$ gcc -W -Wall -c Computer-Security/source/mem_layout.c
$ gcc -o mem_layout mem_layout.o
$ size mem_layout mem_layout.o
```

Why don't we see stack and heap information?

In [9]:
%%writefile source/mem_layout_print.c
#include <stdlib.h>
#include <stdio.h>

int x = 100;
int main()
{
  int   a=2;
  double b=2.5;
  int   c=4;
  static int y;
  int *ptr = (int *) malloc(2*sizeof(int));  
  ptr[0]=5;
  ptr[1]=6;

  printf ("x is %d and is stored at %p\n", x, &x);
  printf ("a is %d and is stored at %p\n", a, &a);
  printf ("b is %f and is stored at %p\n", b, &b);
  printf ("c is %d and is stored at %p\n", c, &c);
  printf ("y is %d and is stored at %p\n", y, &y);
  printf ("ptr[0] is %d and is stored at %p\n", ptr[0], &ptr[0]);
  printf ("ptr[1] is %d and is stored at %p\n", ptr[1], &ptr[1]);

  // deallocate memory on heap	
  free(ptr);
  return 1;
}

Overwriting source/mem_layout_print.c


Slide Type
Run the followings:

```
$ gcc -o mem_layout_print Computer-Security/source/mem_layout_print.c
$ ./mem_layout_print
```

### Stack Memory Layout

- When a function is called, a block of memory called `stack frame` will be pushed onto the top of stack. A **stack frame** contains four regions:
    - Arguments that are passed to the function
    - Return address (the address of the instructions right after the function call
    - Previous frame pointer
    - Local variables of the function
- When the program first starts, the stack contains only one frame, that of the `main` function. 

<center> <img src="figure/buffer-overflow/bo2.png" width="400"/>

In [10]:
%%writefile source/stack_trace.c
#include<stdio.h>
static void display(int i, int *ptr);
    
int main(void) {
 int x = 5;
 int *xptr = &x;
 printf("In main():\n");
 printf("   x is %d and is stored at %p.\n", x, &x);
 printf("   xptr points to %p which holds %d.\n", xptr, *xptr);
 display(x, xptr);
 print ("The display function has stopped");
 return 0;
}
   
void display(int z, int *zptr) {
  printf("In display():\n");
  printf("   z is %d and is stored at %p.\n", z, &z);
  printf("   zptr points to %p which holds %d.\n", zptr, *zptr);
}

Writing source/stack_trace.c


Run the followings:

```
$ gcc -ggdb3 -o stack_trace Computer-Security/source/stack_trace.c
$ ./stack_trace
$ gdb stack_trace
gdb-peda$ 
```

In Class: Alternate between `run`, `break 10`, `backtrace`, `step`, and `n` to see how stack grows. `quit` to quit `gdb`.

```
gdb-peda$ run
gdb-peda$ break 10
gdb-peda$ run
gdb-peda$ backtrace
gdb-peda$ step
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ n
gdb-peda$ n
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ n
gdb-peda$ backtrace
```


<center> <img src="figure/buffer-overflow/bo3.png" width="400"/>

### Buffer Overflow Attack

- Memory copy happens in programming when data from one place (source) is duplicated to another place (destination). 
- Before copying can happen, memory needs to be allocated at the destination. 
- If the allocation fails to be sufficient, it will result in an overflow. 
- One of the oldest and most well-known attacks.

#### Why do we need to learn this vulnerability and attack?

- Can still be found buried in legacy or glue code from third party libraries as web sites get more complex and evolved.
- An important point of learning as you work through the theory and practice of this exploit:
  - C programming
  - C Assembler
  - Linux debugger using gdb
  - Engage in mathematics to understand the breaking points and the hex contents in memory in order to place an attack

**[Smashing the Stack, 2016](https://www.aap3recruitment.com/news/smashing-the-stack/14074/)**

**strcpy()**

In [12]:
%%writefile source/test_strcpy.c
#include <string.h>
#include <stdio.h>
void main(){
  char src[40] = "Hello World \0 Extra string";
  char dest[40];
  strcpy(dest, src);
  printf("%s",dest);
}

Overwriting source/test_strcpy.c


Compile and run `test_strcpy` in your terminal.
- What happens? Why is the string not copied properly?

In [13]:
%%writefile source/strcpy_overflow.c
#include <string.h>

void foo(char *str)
{
    char buffer[12];

    /* The following statement will result in buffer overflow */ 
    strcpy(buffer, str);
}

int main()
{
    char *str = "This is definitely longer than 12";    
    foo(str);

    return 1;
}

Writing source/strcpy_overflow.c


Run the followings:
```
$ gcc -o strcpy_overflow Computer-Security/source/strcpy_overflow.c
$ ./strcpy_overflow
```

Run the followings:
```
$ gcc -z execstack -fno-stack-protector -o strcpy_overflow Computer-Security/source/strcpy_overflow.c
$ ./strcpy_overflow
```

Run the gdb debugging sequence. Examing the frames at each backtrace. What happens to the last backtrace?
```
gdb-peda$ break 14
gdb-peda$ run
gdb-peda$ backtrace
gdb-peda$ step
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ backtrace
```

<center> <img src="figure/buffer-overflow/bo4.png" width="400"/>